In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from  pyspark.sql.functions import input_file_name
from pyspark.sql.functions import col, substring
from pyspark.sql.types import IntegerType

spark = SparkSession.builder.master("local[1]") \
                    .appName('py_spark') \
                    .getOrCreate()

data_df =spark.read.option("header",True).csv("/home/james/python_practice/archive")
data_df.filter(data_df.tags.like("%rap%")).sort("dislikes").show()

NameError: name 'spark' is not defined